# Cliente


## Imports básicos

In [4]:
import requests
import numpy as np
import json

## Definir url de la API

In [8]:
base_url = 'http://localhost:8000'
endpoint = '/predict'
confidence = 0.5

Para consumir el modelo, vamos a agregar el endpoint a la URL base para obtener la URL completa.

In [9]:
url_with_endpoint_no_params = base_url + endpoint
url_with_endpoint_no_params

'http://localhost:8000/predict'

Para agregar los parámetros que deseamos, tenemos que usar la sintaxis de agregar un caracter "?" seguido del nombre del parámetro y su valor.

Si lo hacemos la URL final quedaría en algo como esto:

In [7]:
full_url = url_with_endpoint_no_params + "?confidence=" + str(confidence)
full_url

'http://localhost:8000/predict?confidence=0.5'


## Peticiones

### Función para hacer la petición

In [14]:
def response_from_server(url, item_features, verbose=True):
    """Hace una solicitud POST al servidor y retorna la respuesta.

    Argumentos:
        url (str): URL a la que se envía la consulta.
        item_features (dict): vector de característica del viaje.
        verbose (bool): True si se debe imprimir el estado de la solicitud. False en otro caso.

    Retorna:
        requests.models.Response: Respuesta del servidor.
    """
    response = requests.post(url, data=json.dumps(item_features))
    status_code = response.status_code
    if verbose:
        msg = "Successful request" if status_code == 200 else "There was an error"
        print(msg)
    return response

### Peticiones

Caso 1: hombre de clase baja, 29 años, 2 familiares a bordo, 3 hijos, tarifa en torno a 20 dólarea y salió del puerto Q

In [17]:
item_features =  {
"pclass": 3, 
"sex": "male", 
"age": 29, 
"sibsp": 2, 
"parch": 3, 
"fare": 20, 
"embarked": "Q"
}
prediction = response_from_server(full_url, item_features)
prediction.text

Successful request


'{"predicted_class":"no survived"}'

Caso 2: mujer de primera clase, 45 años, 1 familiar a bordo, 2 hijos, 500 dólares de tarifa y salió del puerto C

In [28]:
item_features =  {
"pclass": 1, 
"sex": "female", 
"age": 29, 
"sibsp": 1, 
"parch": 2, 
"fare": 500, 
"embarked": "C"
}
prediction = response_from_server(full_url, item_features)
prediction.text

Successful request


'{"predicted_class":"survived"}'

Caso 3: ingresar cualquier tipo en "embarked". Esto debe ser error, ya que solo se permite "C", "S" o "Q", donde  C = Cherbourg, Q = Queenstown y S = Southampton

In [26]:
item_features =  {
"pclass": 1, 
"sex": "female", 
"age": 29, 
"sibsp": 1, 
"parch": 2, 
"fare": 500, 
"embarked": "Valparaíso"
}
prediction = response_from_server(full_url, item_features)
prediction.text

There was an error


'{"detail":[{"type":"enum","loc":["body","embarked"],"msg":"Input should be \'C\', \'S\' or \'Q\'","input":"Valparaíso","ctx":{"expected":"\'C\', \'S\' or \'Q\'"}}]}'

Caso 4: se ingresa una tarifa negativa, lo cual debe ser error porque evidentemente el dinero negativo no existe

In [27]:
item_features =  {
"pclass": 1, 
"sex": "female", 
"age": 29, 
"sibsp": 1, 
"parch": 2, 
"fare": -100, 
"embarked": "C"
}
prediction = response_from_server(full_url, item_features)
prediction.text

There was an error


'{"detail":[{"type":"greater_than","loc":["body","fare"],"msg":"Input should be greater than 0","input":-100,"ctx":{"gt":0.0}}]}'

Caso 5: se ingresa una clase que no corresponde (solo puede ser 1, 2 o 3)

In [29]:
item_features =  {
"pclass": 4, 
"sex": "female", 
"age": 29, 
"sibsp": 1, 
"parch": 2, 
"fare": 100, 
"embarked": "C"
}
prediction = response_from_server(full_url, item_features)
prediction.text

There was an error


'{"detail":[{"type":"less_than_equal","loc":["body","pclass"],"msg":"Input should be less than or equal to 3","input":4,"ctx":{"le":3}}]}'

Caso 6: mujer mayor, sin familiares ni hijos, clase baja y desde el puerto C

In [31]:
item_features =  {
"pclass": 3, 
"sex": "female", 
"age": 70, 
"sibsp": 0, 
"parch": 0, 
"fare": 10, 
"embarked": "C"
}
prediction = response_from_server(full_url, item_features)
prediction.text

Successful request


'{"predicted_class":"no survived"}'

Caso 7: la misma mujer del caso 6, pero de clase alta

In [32]:
item_features =  {
"pclass": 1, 
"sex": "female", 
"age": 70, 
"sibsp": 0, 
"parch": 0, 
"fare": 400, 
"embarked": "C"
}
prediction = response_from_server(full_url, item_features)
prediction.text

Successful request


'{"predicted_class":"survived"}'